In [1]:
import numpy as np
import pandas as pd
import os
import json
from scipy import signal, fft, interpolate
from sklearn.decomposition import FastICA
import traceback
from scipy.signal import firwin, filtfilt
from tqdm import tqdm
from scipy.ndimage import label

# =============================
# UPDATED CONFIGURATION
# =============================
BASE_PATH = '/kaggle/input/mtcaic3'
PREPROCESSED_PATH = './improved_preprocessed/'
EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
MOTION_SENSORS = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
FS = 250  # Sampling rate (Hz)
ARTIFACT_REJECTION_THRESHOLD = 40  # Increased threshold to 40%

# =============================
# FIXED HELPER FUNCTIONS
# =============================
def remove_line_noise(data, f0=50, harmonics=3):
    """Advanced multi-notch filtering using FIR filters"""
    filtered = data.copy()
    n_samples = len(data)
    
    # Design high-quality FIR notch filters
    for n in range(1, harmonics + 1):
        freq = n * f0
        if freq < FS/2.1:
            # Create FIR notch filter with better stopband attenuation
            notch_width = 2.0  # Hz
            nyq = FS / 2.0
            freq_range = [freq - notch_width/2, freq + notch_width/2]
            taps = firwin(401, freq_range, fs=FS, pass_zero=False)
            filtered = filtfilt(taps, 1.0, filtered, axis=0)
    
    return filtered

def detect_clipping(channel_data):
    """Improved clipping detection with run-length encoding"""
    diff = np.diff(channel_data, prepend=channel_data[0])
    flat_points = (np.abs(diff) < 1e-4)
    
    # Handle case with no flat points
    if not np.any(flat_points):
        return np.zeros_like(flat_points, dtype=bool)
    
    # Use run-length encoding to find consecutive flat points
    structure = np.ones(3)  # 1D connectivity
    labeled, ncomponents = label(flat_points, structure=structure)
    
    long_flat = np.zeros_like(flat_points, dtype=bool)
    for i in range(1, ncomponents + 1):
        component = (labeled == i)
        run_length = np.sum(component)
        if run_length > 10:
            long_flat[component] = True
    
    return long_flat

def preprocess_trial(trial_data, task, trial_id=None):
    """Enhanced preprocessing with artifact handling"""
    try:
        # 1. Convert nV to μV
        eeg_data = trial_data[EEG_CHANNELS].copy() / 1000
        
        # 2. Extract validation flags and motion sensors
        validation_flags = trial_data['Validation'].values if 'Validation' in trial_data else None
        motion_data = trial_data[MOTION_SENSORS].copy() if all(col in trial_data for col in MOTION_SENSORS) else None
        
        # 3. Improved DC removal using median
        eeg_data = eeg_data - np.median(eeg_data, axis=0)
        
        # 4. Create artifact mask
        artifact_mask = np.zeros(len(eeg_data), dtype=bool)
        artifact_counts = {'validation': 0, 'clipping': 0, 'motion': 0}
        
        # Handle validation flags
        if validation_flags is not None:
            validation_artifacts = (validation_flags == 0)
            artifact_mask |= validation_artifacts
            artifact_counts['validation'] = int(np.sum(validation_artifacts))
        
        # Apply clipping detection per channel
        clipping_mask = np.zeros(len(eeg_data), dtype=bool)
        for channel in EEG_CHANNELS:
            channel_mask = detect_clipping(eeg_data[channel].values)
            clipping_mask |= channel_mask
        
        artifact_mask |= clipping_mask
        artifact_counts['clipping'] = int(np.sum(clipping_mask))
        
        # Motion artifact detection
        if motion_data is not None and not motion_data.empty:
            acc_mag = np.sqrt(motion_data[['AccX', 'AccY', 'AccZ']].pow(2).sum(axis=1))
            gyro_mag = np.sqrt(motion_data[['Gyro1', 'Gyro2', 'Gyro3']].pow(2).sum(axis=1))
            motion_index = acc_mag + gyro_mag
            
            if motion_index.nunique() > 1:
                motion_threshold = np.percentile(motion_index, 95)  # More conservative
                motion_artifacts = motion_index > motion_threshold
                artifact_mask |= motion_artifacts
                artifact_counts['motion'] = int(np.sum(motion_artifacts))
        
        # Calculate artifact percentage
        total_artifacts = np.sum(artifact_mask)
        artifact_percent = total_artifacts / len(artifact_mask) * 100
        
        # ===== QUALITY CONTROL =====
        if artifact_percent > ARTIFACT_REJECTION_THRESHOLD:
            if trial_id:
                print(f"REJECTING trial {trial_id}: {artifact_percent:.1f}% artifacts")
            return None, artifact_percent, artifact_counts
        
        # Log artifact statistics
        if trial_id:
            print(f"Trial {trial_id}: Artifacts - {artifact_percent:.1f}% "
                  f"(Validation: {artifact_counts['validation']}, "
                  f"Clipping: {artifact_counts['clipping']}, "
                  f"Motion: {artifact_counts['motion']})")
        
        # 5. Conservative artifact expansion (25ms)
        expanded_mask = artifact_mask.copy()
        artifact_indices = np.where(artifact_mask)[0]
        
        if len(artifact_indices) > 0:
            for i in artifact_indices:
                start = max(0, i - 6)  # 25ms before
                end = min(len(artifact_mask), i + 7)  # 25ms after
                expanded_mask[start:end] = True
        
        # 6. Advanced interpolation with cubic splines
        valid_mask = ~expanded_mask
        x = np.arange(len(eeg_data))
        
        # Ensure minimum valid points
        min_valid_points = max(100, int(0.15 * len(eeg_data)))
        
        for channel in EEG_CHANNELS:
            channel_data = eeg_data[channel].values
            if np.sum(valid_mask) >= 4:  # Minimum for cubic spline
                # Use cubic spline interpolation
                cs = interpolate.CubicSpline(x[valid_mask], channel_data[valid_mask])
                eeg_data[channel] = cs(x)
            else:
                # Fallback to linear interpolation
                eeg_data[channel] = np.interp(x, x[valid_mask], channel_data[valid_mask])
        
        # 7. Enhanced Line Noise Removal
        eeg_values = remove_line_noise(eeg_data.values)
        eeg_data = pd.DataFrame(eeg_values, columns=EEG_CHANNELS)
        
        # 8. Mu-band preserving filter
        sos_mu = signal.butter(4, [8, 13], btype='bandpass', output='sos', fs=FS)
        mu_power = signal.sosfiltfilt(sos_mu, eeg_data.values, axis=0)
        
        # 9. Broadband filter for general processing
        sos_bp = signal.butter(4, [4, 40], btype='bandpass', output='sos', fs=FS)
        filtered = signal.sosfiltfilt(sos_bp, eeg_data.values, axis=0)
        
        # 10. Motion artifact handling - only when necessary
        if motion_data is not None and artifact_counts['motion'] > 100:
            try:
                ica = FastICA(n_components=len(EEG_CHANNELS), random_state=0, max_iter=1000)
                ica_sources = ica.fit_transform(filtered)
                
                # Compute motion correlations
                motion_corr = np.zeros(ica_sources.shape[1])
                for i in range(ica_sources.shape[1]):
                    corr = np.corrcoef(np.abs(ica_sources[:, i]), 
                                      np.abs(motion_data.mean(axis=1)))[0, 1]
                    motion_corr[i] = np.abs(corr)
                
                # Remove only strongly correlated components (threshold=0.6)
                motion_components = motion_corr > 0.6
                if np.any(motion_components):
                    ica_sources[:, motion_components] = 0
                    filtered = ica.inverse_transform(ica_sources)
            except Exception as e:
                if trial_id:
                    print(f"ICA skipped in trial {trial_id}: {str(e)}")
        
        # 11. Mu-band enhancement
        filtered = 0.7 * filtered + 0.3 * mu_power
        
        return pd.DataFrame(filtered, columns=EEG_CHANNELS), artifact_percent, artifact_counts
    
    except Exception as e:
        print(f"Error during preprocessing trial {trial_id}: {str(e)}")
        traceback.print_exc()
        return None, None, None

def load_full_trial(row):
    """Load complete trial data with all columns"""
    try:
        # Determine dataset split
        id_num = row['id']
        if id_num <= 4800:
            split = 'train'
        elif id_num <= 4900:
            split = 'validation'
        else:
            split = 'test'
        
        # Build file path
        eeg_path = f"{BASE_PATH}/{row['task']}/{split}/{row['subject_id']}/{row['trial_session']}/EEGdata.csv"
        
        # Load EEG data
        full_data = pd.read_csv(eeg_path)
        
        # Calculate trial indices
        trial_duration = 2250 if row['task'] == 'MI' else 1750
        start_idx = (row['trial'] - 1) * trial_duration
        end_idx = start_idx + trial_duration
        
        # Handle short trials
        if end_idx > len(full_data):
            end_idx = len(full_data)
            if end_idx <= start_idx:
                raise ValueError(f"Invalid trial range: {start_idx}-{end_idx}")
        
        return full_data.iloc[start_idx:end_idx].copy()
    
    except Exception as e:
        print(f"Error loading trial {row['id']}: {str(e)}")
        raise

def save_preprocessed_data(preprocessed_data, row):
    """Save preprocessed data to organized directory structure as CSV"""
    # Determine dataset split
    id_num = row['id']
    if id_num <= 4800:
        split = 'train'
    elif id_num <= 4900:
        split = 'validation'
    else:
        split = 'test'
    
    # Create output directory
    output_dir = f"{PREPROCESSED_PATH}/{row['task']}/{split}/{row['subject_id']}/{row['trial_session']}/"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save as CSV file with header
    output_path = f"{output_dir}/trial_{row['trial']}.csv"
    preprocessed_data.to_csv(output_path, index=False)
    
    return output_path

def load_preprocessed_trial(row):
    """Load preprocessed EEG data for a single trial from CSV file"""
    # Skip trials that were rejected or had errors
    if 'processed_path' not in row or not isinstance(row['processed_path'], str) or pd.isna(row['processed_path']):
        return None
    
    try:
        return pd.read_csv(row['processed_path'])
    except Exception as e:
        print(f"Error loading preprocessed trial {row['id']}: {e}")
        return None

def analyze_dataset(df, dataset_name, quality_report=None):
    """Analyze preprocessed dataset quality"""
    # Initialize reports
    noise_report = {
        'total_trials': len(df),
        'processed_trials': 0,
        'noisy_percentage': 0,
    }
    
    rejection_info = {
        'rejected_trials': [],
        'error_trials': [],
        'rejection_rate': 0,
        'mean_artifact_percent': 0
    }
    
    # Count processed trials
    processed_trials = 0
    total_artifact_percent = 0
    
    for idx, row in df.iterrows():
        if pd.notna(row['processed_path']):
            processed_trials += 1
            if 'artifact_percent' in row and pd.notna(row['artifact_percent']):
                total_artifact_percent += row['artifact_percent']
        else:
            rejection_info['rejected_trials'].append(row['id'])
    
    # Calculate metrics
    noise_report['processed_trials'] = processed_trials
    if processed_trials > 0:
        rejection_info['mean_artifact_percent'] = total_artifact_percent / processed_trials
    rejection_info['rejection_rate'] = (len(rejection_info['rejected_trials']) / len(df)) * 100
    
    return {
        'dataset': dataset_name,
        'noise_report': noise_report,
        'rejection_info': rejection_info
    }

# =============================
# UPDATED PIPELINE EXECUTION
# =============================
if __name__ == '__main__':
    # Create output directory
    os.makedirs(PREPROCESSED_PATH, exist_ok=True)
    
    # Load datasets
    train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
    val_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))
    test_df = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
    
    # Preprocess all datasets
    results = []
    for name, df in [('train', train_df), ('validation', val_df), ('test', test_df)]:
        print(f"\n{'='*50}")
        print(f"Processing {name} dataset")
        print(f"{'='*50}")
        
        processed_paths = []
        artifact_percents = []
        artifact_counts_list = []
        
        # Process each trial
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {name}"):
            try:
                # Load raw trial data
                trial_data = load_full_trial(row)
                
                # Preprocess trial
                preprocessed, artifact_percent, artifact_counts = preprocess_trial(
                    trial_data, row['task'], row['id']
                )
                
                # Save and record results
                if preprocessed is not None:
                    path = save_preprocessed_data(preprocessed, row)
                    processed_paths.append(path)
                    artifact_percents.append(artifact_percent)
                    artifact_counts_list.append(artifact_counts)
                else:
                    processed_paths.append(None)
                    artifact_percents.append(artifact_percent)
                    artifact_counts_list.append(artifact_counts)
                    
            except Exception as e:
                print(f"Error processing trial {row['id']}: {str(e)}")
                processed_paths.append(None)
                artifact_percents.append(None)
                artifact_counts_list.append(None)
        
        # Add new columns to dataframe
        df['processed_path'] = processed_paths
        df['artifact_percent'] = artifact_percents
        df['artifact_counts'] = artifact_counts_list
        
        # Save updated dataset
        df.to_csv(os.path.join(PREPROCESSED_PATH, f'{name}.csv'), index=False)
        
        # Analyze dataset
        results.append(analyze_dataset(df, name))
    
    # Display results
    print("\n\nPREPROCESSING SUMMARY")
    print("=" * 60)
    for res in results:
        print(f"\n===== {res['dataset'].upper()} SET RESULTS =====")
        print(f"Total trials: {res['noise_report']['total_trials']}")
        print(f"Processed trials: {res['noise_report']['processed_trials']}")
        print(f"Rejected trials: {len(res['rejection_info']['rejected_trials'])}")
        print(f"Rejection rate: {res['rejection_info']['rejection_rate']:.2f}%")
        print(f"Mean artifact %%: {res['rejection_info']['mean_artifact_percent']:.2f}%")
        print("=" * 60)


Processing train dataset


Processing train:   0%|          | 0/4800 [00:00<?, ?it/s]

Trial 1: Artifacts - 8.2% (Validation: 72, Clipping: 64, Motion: 113)


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
Processing train:   0%|          | 1/4800 [00:00<1:17:29,  1.03it/s]/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(


Trial 2: Artifacts - 5.0% (Validation: 0, Clipping: 0, Motion: 113)


...

Trial 4997: Artifacts - 5.0% (Validation: 0, Clipping: 0, Motion: 88)
Trial 4998: Artifacts - 5.0% (Validation: 0, Clipping: 0, Motion: 88)


Processing test:  99%|█████████▉| 99/100 [00:44<00:00,  6.26it/s]

Trial 4999: Artifacts - 5.0% (Validation: 0, Clipping: 0, Motion: 88)
Trial 5000: Artifacts - 5.0% (Validation: 0, Clipping: 0, Motion: 88)


Processing test: 100%|██████████| 100/100 [00:45<00:00,  2.22it/s]



PREPROCESSING SUMMARY

===== TRAIN SET RESULTS =====
Total trials: 4800
Processed trials: 4653
Rejected trials: 147
Rejection rate: 3.06%
Mean artifact %%: 6.07%

===== VALIDATION SET RESULTS =====
Total trials: 100
Processed trials: 98
Rejected trials: 2
Rejection rate: 2.00%
Mean artifact %%: 9.10%

===== TEST SET RESULTS =====
Total trials: 100
Processed trials: 100
Rejected trials: 0
Rejection rate: 0.00%
Mean artifact %%: 7.55%


In [2]:
# import numpy as np
# import pandas as pd
# import os
# from scipy import stats, signal
# import json

# # Configuration - Point to your preprocessed data directory
# BASE_PATH = '/kaggle/working/improved_preprocessed/'
# EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
# FS = 250  # Sampling rate (Hz)

# # Detection thresholds (adjusted for cleaned data)
# FLATLINE_THRESHOLD = 1e-3    # Variance threshold for flatline detection (uV²)
# ABNORMAL_VOLTAGE = 100       # Absolute voltage threshold (uV)
# RANGE_THRESHOLD = 80         # Peak-to-peak difference threshold (uV)
# KURTOSIS_THRESHOLD = 10      # Threshold for abnormal distribution peakedness
# LINE_NOISE_THRESHOLD = 3     # SNR ratio for line noise detection
# MU_BAND = (8, 13)            # Mu band frequency range (Hz)

# def load_preprocessed_trial(row):
#     """Load preprocessed EEG data for a single trial from .npz file"""
#     # Skip trials that were rejected or had errors
#     if not isinstance(row['processed_path'], str) or pd.isna(row['processed_path']):
#         return None
        
#     try:
#         # Load from the processed_path column added during preprocessing
#         data = np.load(row['processed_path'])['data']
#         return pd.DataFrame(data, columns=EEG_CHANNELS)
#     except Exception as e:
#         print(f"Error loading trial {row['id']}: {str(e)}")
#         return None

# def detect_line_noise(channel_data, target_freq):
#     """Detect line noise at specific frequency using SNR ratio"""
#     # Compute power spectral density
#     try:
#         f, psd = signal.welch(channel_data, FS, nperseg=256)
        
#         # Find target frequency bin
#         target_idx = np.argmin(np.abs(f - target_freq))
#         target_power = psd[target_idx]
        
#         # Define surrounding noise bands
#         low_band = (target_freq - 5, target_freq - 1)
#         high_band = (target_freq + 1, target_freq + 5)
        
#         # Calculate average noise power
#         noise_mask = ((f >= low_band[0]) & (f <= low_band[1])) | ((f >= high_band[0]) & (f <= high_band[1]))
#         noise_power = np.mean(psd[noise_mask])
        
#         # Avoid division by zero
#         if noise_power < 1e-10:
#             return 0
        
#         # Compute SNR ratio
#         return target_power / noise_power
#     except:
#         return 0

# def detect_noise(eeg_data):
#     """Detect various noise artifacts in EEG data"""
#     if eeg_data is None:
#         return {}, []
    
#     noise_flags = {
#         'flat_channels': [],
#         'abnormal_voltage': [],
#         'high_range': [],
#         'high_kurtosis': [],
#         'line_noise_50Hz': [],
#         'line_noise_60Hz': [],
#         'variance_outliers': []
#     }
    
#     # Channel-wise analysis
#     channel_metrics = []
#     for channel in EEG_CHANNELS:
#         channel_data = eeg_data[channel].values
        
#         # 1. Statistical metrics
#         variance = np.var(channel_data)
#         data_range = np.ptp(channel_data)  # Peak-to-peak (max-min)
#         kurt = stats.kurtosis(channel_data)
        
#         # 2. Flatline detection
#         if variance < FLATLINE_THRESHOLD:
#             noise_flags['flat_channels'].append(channel)
        
#         # 3. Abnormal voltage detection
#         if np.max(np.abs(channel_data)) > ABNORMAL_VOLTAGE:
#             noise_flags['abnormal_voltage'].append(channel)
            
#         # 4. Range detection
#         if data_range > RANGE_THRESHOLD:
#             noise_flags['high_range'].append(channel)
            
#         # 5. Kurtosis detection
#         if kurt > KURTOSIS_THRESHOLD:
#             noise_flags['high_kurtosis'].append(channel)
            
#         # 6. Line noise detection
#         snr_50 = detect_line_noise(channel_data, 50)
#         snr_60 = detect_line_noise(channel_data, 60)
        
#         if snr_50 > LINE_NOISE_THRESHOLD:
#             noise_flags['line_noise_50Hz'].append(channel)
#         if snr_60 > LINE_NOISE_THRESHOLD:
#             noise_flags['line_noise_60Hz'].append(channel)
            
#         # Store metrics for variance outlier detection
#         channel_metrics.append({
#             'channel': channel,
#             'variance': variance,
#             'range': data_range,
#             'kurtosis': kurt,
#             'snr_50': snr_50,
#             'snr_60': snr_60
#         })
    
#     # 7. Variance outlier detection using IQR
#     if channel_metrics:
#         variances = [m['variance'] for m in channel_metrics]
#         q1, q3 = np.percentile(variances, [25, 75])
#         iqr = q3 - q1
#         for i, metrics in enumerate(channel_metrics):
#             if metrics['variance'] < (q1 - 1.5*iqr) or metrics['variance'] > (q3 + 1.5*iqr):
#                 noise_flags['variance_outliers'].append(metrics['channel'])
    
#     return noise_flags, channel_metrics

# def check_contralateral_pattern(eeg_data, label):
#     """Verify contralateral pattern in MI trials"""
#     if eeg_data is None or label not in ['Left', 'Right']:
#         return False
        
#     c3_data = eeg_data['C3'].values
#     c4_data = eeg_data['C4'].values
    
#     # Bandpower calculation (mu rhythm)
#     f, psd = signal.welch([c3_data, c4_data], FS, nperseg=256)
#     mu_band = (f >= MU_BAND[0]) & (f <= MU_BAND[1])
    
#     c3_power = np.mean(psd[0][mu_band])
#     c4_power = np.mean(psd[1][mu_band])
    
#     # Contralateral pattern check
#     if label == 'Left':
#         return c4_power < c3_power  # Right hemisphere (C4) should show ERD
#     elif label == 'Right':
#         return c3_power < c4_power  # Left hemisphere (C3) should show ERD
#     return False

# def analyze_dataset(df, dataset_name, quality_report=None):
#     """Run full analysis on a dataset"""
#     # Initialize noise counters
#     noise_types = [
#         'flat_channels', 'abnormal_voltage', 'high_range', 
#         'high_kurtosis', 'line_noise_50Hz', 'line_noise_60Hz',
#         'variance_outliers'
#     ]
#     noise_report = {ntype: 0 for ntype in noise_types}
#     noise_report['total_trials'] = len(df)
#     noise_report['noisy_trials'] = 0
#     noise_report['processed_trials'] = 0
    
#     # Initialize metric aggregators
#     metric_agg = {
#         'variance': [],
#         'range': [],
#         'kurtosis': [],
#         'snr_50': [],
#         'snr_60': []
#     }
    
#     contralateral_counts = {'total_mi': 0, 'valid_pattern': 0}
    
#     for _, row in df.iterrows():
#         # Load preprocessed EEG data
#         eeg_data = load_preprocessed_trial(row)
        
#         if eeg_data is None:
#             continue  # Skip rejected or errored trials
            
#         noise_report['processed_trials'] += 1
        
#         # Noise detection
#         noise_flags, channel_metrics = detect_noise(eeg_data)
#         if any(noise_flags.values()):
#             noise_report['noisy_trials'] += 1
#             for ntype in noise_types:
#                 if noise_flags[ntype]:
#                     noise_report[ntype] += len(noise_flags[ntype])
        
#         # Aggregate metrics across channels
#         for metric in metric_agg:
#             metric_agg[metric].extend([m[metric] for m in channel_metrics])
        
#         # Contralateral pattern (MI only)
#         if 'label' in row and row['task'] == 'MI':
#             contralateral_counts['total_mi'] += 1
#             if check_contralateral_pattern(eeg_data, row['label']):
#                 contralateral_counts['valid_pattern'] += 1
    
#     # Calculate percentages
#     if noise_report['processed_trials'] > 0:
#         noise_report['noisy_percentage'] = (noise_report['noisy_trials'] / noise_report['processed_trials']) * 100
#     else:
#         noise_report['noisy_percentage'] = 0
    
#     if contralateral_counts['total_mi'] > 0:
#         contralateral_counts['pattern_percentage'] = (contralateral_counts['valid_pattern'] / contralateral_counts['total_mi']) * 100
#     else:
#         contralateral_counts['pattern_percentage'] = 0
    
#     # Calculate metric statistics
#     metric_stats = {}
#     for metric, values in metric_agg.items():
#         if values:  # Only calculate if we have data
#             metric_stats[metric] = {
#                 'mean': np.mean(values),
#                 'median': np.median(values),
#                 'q1': np.percentile(values, 25),
#                 'q3': np.percentile(values, 75)
#             }
    
#     # Add quality report information if available
#     rejection_info = {}
#     if quality_report:
#         rejection_info = {
#             'rejected_trials': quality_report.get('rejected_trials', []),
#             'error_trials': quality_report.get('error_trials', []),
#             'rejection_rate': quality_report.get('rejection_rate', 0),
#             'mean_artifact_percent': quality_report.get('mean_artifact_percent', 0)
#         }
    
#     return {
#         'dataset': dataset_name,
#         'noise_report': noise_report,
#         'metric_stats': metric_stats,
#         'contralateral_report': contralateral_counts,
#         'rejection_info': rejection_info
#     }

# # Main analysis
# if __name__ == '__main__':
#     # Load datasets with processed_path column
#     train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
#     val_df = pd.read_csv(os.path.join(BASE_PATH, 'validation.csv'))
#     test_df = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
    
#     # Load quality reports
#     quality_reports = {}
#     for name in ['train', 'validation', 'test']:
#         try:
#             with open(os.path.join(BASE_PATH, f"{name}_quality.json")) as f:
#                 quality_reports[name] = json.load(f)
#         except:
#             print(f"Warning: Could not load quality report for {name}")
#             quality_reports[name] = {}
    
#     # Run analysis
#     results = []
#     for df, name in [(train_df, 'train'), (val_df, 'validation'), (test_df, 'test')]:
#         print(f"Analyzing {name} set...")
#         results.append(analyze_dataset(df, name, quality_reports.get(name)))
    
#     # Display results
#     for res in results:
#         print(f"\n===== {res['dataset'].upper()} SET RESULTS =====")
        
#         # Show rejection info if available
#         if res['rejection_info']:
#             print(f"Total trials: {res['noise_report']['total_trials']}")
#             print(f"Rejected trials: {len(res['rejection_info'].get('rejected_trials', []))}")
#             print(f"Error trials: {len(res['rejection_info'].get('error_trials', []))}")
#             print(f"Rejection rate: {res['rejection_info'].get('rejection_rate', 0):.2f}%")
#             print(f"Mean artifact %% in processed: {res['rejection_info'].get('mean_artifact_percent', 0):.2f}%")
        
#         print(f"\nProcessed trials: {res['noise_report']['processed_trials']}")
#         print(f"Noisy trials: {res['noise_report']['noisy_percentage']:.2f}%")
        
#         print("\nNoise Type Counts (across all channels in processed trials):")
#         for ntype in [
#             'flat_channels', 'abnormal_voltage', 'high_range', 
#             'high_kurtosis', 'line_noise_50Hz', 'line_noise_60Hz',
#             'variance_outliers'
#         ]:
#             print(f"- {ntype}: {res['noise_report'][ntype]} channels")
        
#         print("\nSignal Metrics (across all channels in processed trials):")
#         for metric, stats in res['metric_stats'].items():
#             print(f"- {metric}: mean={stats['mean']:.4f}, median={stats['median']:.4f}")
        
#         if res['contralateral_report']['total_mi'] > 0:
#             print(f"\nContralateral pattern adherence: {res['contralateral_report']['pattern_percentage']:.2f}% of MI trials")
        
#         print("=" * 60)